In [32]:
from pythreejs import *
#from .Py3dv_mesh import Py3dv_mesh
#from .Py3dv_tetrahedron import Py3dv_tetrahedron
#from .Py3dv_triangle import Py3dv_triangle
from py3Dviewer import *
import numpy as np
import math as mt
import timeit
from functools import reduce


class Py3dv_triMesh(Py3dv_mesh):
    
    def __init__(self, filename = None, vertices = None, faces = None):
        
        """
        Class that represents a triangular mesh
        Parameters
        ----------
        filename : String
           name of the file to open
        vertices : Array
           An array of shape N x 3
        faces : Array
           An array of shape N x 3
        Return
        ------
        A trimesh object
        """
        
        self.tri2index = {} #dictionary that returns the index of a triangle given its coord Ids
        self.__numTri = 0
        self.__trianglesSet = set()
        self.triList = [] #lista di MyTriangles
        self.npTriList  = None
        self.surface = None
        super().__init__()
        
        if filename is not None:
            
            self.loadMeshFromFile(filename)
        
        elif vertices is not None and faces is not None:
            self.loadMeshFromValues(vertices, faces)
        
        else:
            pass
        
    def __getattribute__(self,name):
        if name=='surface' and self.isDirty:
            self.__updateSurface()
            return self.surface
        elif name=='npTriList':
            return np.array([[t.idV0, t.idV1, t.idV2] for t in self.triList])
        else:
            return object.__getattribute__(self, name)
        
    def triangleExists(self, triangle):

        return (triangle[0], triangle[1], triangle[2]) in self.__trianglesSet
        
        
    def addTriangle(self, triangle):

        """
        Add an object of type "triangle" into the current mesh
        Parameters
        ----------
        triangle : Triangle
           The triangle that will be added
        """
        
        self.triList[self.__numTri] = triangle
        self.tri2index[triangle.idV0,triangle.idV1,triangle.idV2] = self.__numTri
        self.__numTri += 1
        self.isDirty = True
    
    def addTriangleFromVtxIds(self, v0, v1, v2):
        """
        Add an object of type "triangle" into the current mesh
        Parameters
        ----------
        v0, v1, v2 : Int
           Vertices IDs composing the new triangle 
        """
        self.triList.append(Py3dv_triangle(v0, v1, v2, self.__numTri))
        self.tri2index[v0,v1,v2] = self.__numTri
        self.__numTri += 1
        self.isDirty = True
    
    def getNumTriangles(self):
        return self.__numTri

    
    # legge file mesh di triangoli
    def loadMeshFromFile(self, nameFile):

        file = open(nameFile,"r") # default
        #cicla sino alla ricerca dei vertici
        
        rows = file.readlines()
        nVertices = 0
        vtxTmp = []
        for row in rows:
            
            if row[0:2] == 'v ':
                
                vtx = list(map(float,row.split()[1:]))
                vtxTmp.append(Py3dv_vertex(vtx[0], vtx[1], vtx[2], nVertices))
                nVertices +=1
            
            if row[0:2] == 'f ':
                
                tri =  list(map(int,row.split()[1:]))
                tri = [i-1 for i in tri]
                self.addTriangleFromVtxIds(tri[0], tri[1], tri[2])
                tri = [(tri[0], tri[1], tri[2])]
                self.__trianglesSet.update(set((tri)))
        
        
        self.vtxList = vtxTmp
        self.npVertices = np.array([v.xyz() for v in self.vtxList])
        self.__numVtx = nVertices
        self.__initSlices()

        print('Read:',self.__numVtx,'vertices')
        print('Read:',self.__numTri,'faces')
        
        
    def loadMeshFromValues(self, vertices, faces):
        
        vertices = np.array(vertices)
        faces = np.array(faces)
         
        try:
            
            if vertices.shape[1] !=3 or faces.shape[1] !=3:
                raise Exception()
            
            nVertices = 0
            vtxTmp = []
            for vtx in vertices:
                vtxTmp.append(Py3dv_vertex(vtx[0], vtx[1], vtx[2], nVertices))
                nVertices +=1
                
            for tri in faces:
                self.addTriangleFromVtxIds(tri[0], tri[1], tri[2])
                tri = [(tri[0], tri[1], tri[2])]
                self.__trianglesSet.update(set((tri)))
            
            self.vtxList = vtxTmp
            self.npVertices = np.array([v.xyz() for v in self.vtxList])
            self.__numVtx = nVertices
            self.__initSlices()
            
        except Exception:
            print('Vertices and faces must be array-like structures of shape [Nx3]')
        
    
    def __initSlices(self):
        
        min_ = self.npVertices.min(axis = 0)
        max_ = self.npVertices.max(axis = 0)
        
        
        self.minX = min_[0]
        self.minY = min_[1]
        self.minZ = min_[2]
        self.maxX = max_[0]
        self.maxY = max_[1]
        self.maxZ = max_[2]
        
        self.setSliceFromX(self.minX)
        self.setSliceFromY(self.minY)
        self.setSliceFromZ(self.minZ)
        self.setSliceToX(self.maxX)
        self.setSliceToY(self.maxY)
        self.setSliceToZ(self.maxZ)
    
    
    def updateSurface(self, flipX=None, flipY=None, flipZ=None):
        
        self.isDirty = False

        self.surface = []
        
        baricenters = np.sum(mesh.npVertices[mesh.npTriList], axis=1)/3
        
        iminx = np.argwhere(baricenters[:,0] > self.fromX)
        imaxx = np.argwhere(baricenters[:,0] < self.toX)
        iminy = np.argwhere(baricenters[:,1] > self.fromY)
        imaxy = np.argwhere(baricenters[:,1] < self.toY)
        iminz = np.argwhere(baricenters[:,2] > self.fromZ)
        imaxz = np.argwhere(baricenters[:,2] < self.toZ)
        
        #TODO: Implement flip...
        
        surface_indices = reduce(np.intersect1d, (iminx, imaxx, iminy, imaxy, iminz, imaxz))
        self.surface = self.npVertices[self.npTriList[surface_indices].flatten()]
        
            
    def __updateDrawableElements(self):
        
        if self.isDirty:
            self.updateSurface()
        
        if not hasattr(self, 'color'):
            self.color = np.array([[0.0,0,1],[0.0,0,1],[0.0,0,1]])
            self.color = np.repeat(self.color, self.__numTri*3, axis=0)

       
    
    #visualizza la mesh
    def show(self, *args):
        self.__updateDrawableElements()
        renderer = self.initializeCamera(list(self.centerPoint()))
        self.__draw()
        display(renderer)

    #crea la mesh
    def __draw(self): 
        startDraw = timeit.default_timer()
        
        triProperties = {
            'position': BufferAttribute(self.surface, normalized=False),
            #'index' : BufferAttribute(np.asarray(self.surface, dtype='uint32').ravel(), normalized=False),
            'color' : BufferAttribute(self.color, normalized=False),
        }

        meshGeometry = BufferGeometry(attributes=triProperties)
    #    meshGeometry.faceColors = [[test.color[i] for i in f] for f in triangleSurface] 
        meshGeometry.exec_three_obj_method('computeVertexNormals')

        meshMaterial = MeshLambertMaterial(#shininess=25,
                                         #emissive = '#aaaaaa',#phong
                                         #specular = '#aaaaaa',#phong
                                           polygonOffset=True,
                                           polygonOffsetFactor=1,
                                           polygonOffsetUnits=1,
                                           flatShading = True,
                                           side = 'FrontSide',
                                           #color = '#550000',
                                           wireframe=False,
                                           vertexColors = 'FaceColors',
                                          )
        edges_material = MeshBasicMaterial(color='black',
#                                           side= 'FrontSide'
                                           polygonOffset=True,
                                           polygonOffsetFactor=1,
                                           polygonOffsetUnits=1,
                                           #shininess=0.5,
                                           wireframe=True,
                                           linewidth = 1,
                                           opacity=1,
                                           depthTest=True,
                                           transparent=True)

        self.mesh = Mesh(
            geometry=meshGeometry,
            material=meshMaterial,
            position=[0, 0, 0]   # Center in 0
        )
        self.line = Mesh(
            geometry=meshGeometry,
            material=edges_material,
            position=[0, 0, 0]   # Center in 0
        )


        #aggiunge la mesh alla scena
        self.scene.add(self.mesh)
        self.scene.add(self.line)


        stopDraw = timeit.default_timer()
        print('Print time: ', round(stopDraw-startDraw,2),'s', '(',int(len(self.surface)/3),' faces)')

        return meshGeometry 
   

    def changeColorSurface(self,color=None,colorHex=None):
         
        """
        Change the color of the mesh surface
        Parameters
        ----------
        color : Array
           Color expressed as [R, G, B] with range [0, 1]
        colorHex : Array
           Color expressed as Hexadecimal value
        """
        if colorHex is not None:
            start = timeit.default_timer()
            color = [int(colorHex[1:3],16)/255.0,int(colorHex[3:5],16)/255.0,int(colorHex[5:7],16)/255.0]
            print(color)
        
        self.color = np.array(color, dtype = np.float)
        self.color = np.repeat(self.color, self.__numTri*3, axis=0)
   
    
    def saveFile(self,fileName, overwrite=False):
       
        """
        Save the current mesh into a file
        Parameters
        ----------
        fileName : String
           Name of the file to save
        overwrite : Bool
           True if you want to overwrite a file
        """        
        if overwrite:
            write='w'
        else:
            write='x'
            
        try:
            f = open(fileName,write)
            for i in self.vtxList:
                f.write('v '+str(i.xyz()[0]) + ' ' + str(i.xyz()[1]) + ' ' + str(i.xyz()[2]) + ' ' + '0\r')
            for i in self.triList:
                f.write('f '+str(i.idV0+1) + ' ' + str(i.idV1+1) + ' ' + str(i.idV2+1) + '\r')
            f.close()
        except FileExistsError:
            print('A file with this name already exists!')
            print('Set the overwrite parameter to True to proceed anyway')

In [33]:
mesh = Py3dv_triMesh('274.obj')

Read: 15772 vertices
Read: 31540 faces


In [34]:
Py3dv_viewer(mesh)

Accordion(children=(VBox(children=(VBox(children=(HBox(children=(FloatRangeSlider(value=(-0.771, 0.818), descr…

-0.769255
-0.769255
Print time:  0.07 s ( 31538  faces)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(0.0, 10.0, 30.0), quater…

In [28]:
mesh.setSliceFromX(-0.5)

In [15]:
mesh.surface.shape[0]/3

31540.0

In [27]:
mesh.toX

0.325

In [35]:
mesh.updateSurface(flipX=True)

-0.406
0.269


In [36]:
mesh.show()

ZeroDivisionError: division by zero

In [22]:
baricenters = np.sum(mesh.npVertices[mesh.npTriList], axis=1)/3

In [35]:
iminx = np.argwhere(baricenters[:,0] < mesh.fromX)
imaxx = np.argwhere(baricenters[:,0] < mesh.toX)
iminy = np.argwhere(baricenters[:,1] < mesh.fromY)
imaxy = np.argwhere(baricenters[:,1] < mesh.toY)
iminz = np.argwhere(baricenters[:,2] < mesh.fromZ)
imaxz = np.argwhere(baricenters[:,2] < mesh.toZ)

In [39]:
np.unique(np.concatenate([iminx, imaxx, iminy, imaxy, iminz, imaxz])).shape

(30726,)